In [1]:
#|default_exp toolloop

# Tool Loop

The code for Claudette's tool loop should essentially work as is. We'll replicate the whole original notebook just to make sure

In [84]:
#| export

from gaspare.core import *
from fastcore.utils import *
from fastcore.meta import delegates

from google import genai

In [85]:
model = models[2]
model

'gemini-2.0-flash'

Let's use the [same example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) from Claudette's documentation.

In [86]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

As with Claudette, we do not have to create the JSON schema manually. We can use docments.

In [69]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We are ready to go. The main difference here is that we don't assign the tools to the chat itself, since otherwise Gemini becomes too eager to use them.

In [6]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model)

In [7]:
r = chat('Can you tell me the email address for customer C1?', tools=tools, use_afc=False)
r

- Retrieving customer C1


<ul><li><code>get_customer_info(customer_id=C1)</code></li></ul>
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>args</code>: <ul><li><b>customer_id</b>: C1</li></ul></li><li><code>name</code>: get_customer_info</li></ul></li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.00010187082225456833</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 147; Out: 10; Total: 157</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [8]:
chat()

The email address for customer C1 is john@example.com.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The email address for customer C1 is john@example.com.
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -2.561106230132282e-05</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 71; Out: 15; Total: 86</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [82]:
sp = """You will be provided with tools, but don't limit your answer to those tools.
If the user query is related to some of the tools you have access to come up with a sequence of actions to achieve the goal and **execute the plan immediately**.

If the user query is unrelated to the tools you have access to, answer the query using your own knowledge."""

chat = Chat(model)
r = chat('Cancel all orders for customer C1', tools=tools, use_afc=False, sp=sp, temp=0.6)
r

Here are some ideas for things to do with your dog on a rainy day:<br /><br />*   **Indoor games:** Play fetch in a hallway, try hide-and-seek with treats, or work on some new training tricks.<br />*   **Puzzle toys:** These can keep your dog mentally stimulated and entertained.<br />*   **Grooming session:** Rainy days are a good time to brush your dog, trim their nails, or clean their ears.<br />*   **Movie time:** Cuddle up on the couch and watch a dog-friendly movie together.<br />*   **Visit a pet-friendly store:** Many pet supply stores allow dogs, so you can take a trip to pick out a new toy or treat.<br />*   **Indoor playdate:** If your dog gets along with other dogs, invite a friend over for an indoor playdate.<br />*   **Dog-friendly cafe:** Some cafes allow dogs inside, so you can enjoy a coffee while your dog relaxes by your side.<br />*   **Short, leashed walks:** If your dog needs to go outside, take them for a short, leashed walk in a covered area or put them in a raincoat. Dry them off thoroughly when you get back.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Here are some ideas for things to do with your dog on a rainy day:

*   **Indoor games:** Play fetch in a hallway, try hide-and-seek with treats, or work on some new training tricks.
*   **Puzzle toys:** These can keep your dog mentally stimulated and entertained.
*   **Grooming session:** Rainy days are a good time to brush your dog, trim their nails, or clean their ears.
*   **Movie time:** Cuddle up on the couch and watch a dog-friendly movie together.
*   **Visit a pet-friendly store:** Many pet supply stores allow dogs, so you can take a trip to pick out a new toy or treat.
*   **Indoor playdate:** If your dog gets along with other dogs, invite a friend over for an indoor playdate.
*   **Dog-friendly cafe:** Some cafes allow dogs inside, so you can enjoy a coffee while your dog relaxes by your side.
*   **Short, leashed walks:** If your dog needs to go outside, take them for a short, leashed walk in a covered area or put them in a raincoat. Dry them off thoroughly when you get back.
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.12269277954101562</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 199; Out: 250; Total: 449</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [87]:
r = chat('Cancel all orders for customer C1', tools=tools, use_afc=False, sp=sp, temp=0.6)
r

- Retrieving customer C1


<ul><li><code>get_customer_info(customer_id=C1)</code></li></ul>
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>args</code>: <ul><li><b>customer_id</b>: C1</li></ul></li><li><code>name</code>: get_customer_info</li></ul></li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.09488453269004822</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 848; Out: 10; Total: 858</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [88]:
#| exports

@patch
@delegates(genai.chats.Chat.__call__)
def toolloop(self:genai.chats.Chat,
             pr, # Prompt to pass to Gemini
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Gemini, automatically following up with `tool_use` messages"
    n_msgs = len(self.h)
    kwargs["use_afc"] = False
    r = self(pr, **kwargs)
    for i in range(max_steps):
        if not r.function_calls:break
        if trace_func: trace_func(self.h[n_msgs:]); n_msgs = len(self.h)
        r = self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(self.h[n_msgs:])
    return r

In [95]:
chat = Chat(model)
r = chat.toolloop('Tell me the email address for customer C1', tools=tools, sp=sp, temp)
r

I do not have the ability to retrieve a customer's email address with the available tools. I can only retrieve customer information and their orders based on the customer ID. Would you like me to get the customer information for customer C1?<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I do not have the ability to retrieve a customer's email address with the available tools. I can only retrieve customer information and their orders based on the customer ID. Would you like me to get the customer information for customer C1?
</li></ul></details></li><li><code>role</code>: model</li></ul></li><li><code>avg_logprobs</code>: -0.05460302197203344</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 219; Out: 49; Total: 268</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [85]:
def print_msgs(msgs):
    for n, m in enumerate(msgs):
        for i, part in enumerate(m.parts):
            print(f"\nMessage {n+1}, Part {i + 1}:\n")
            c = "* Text *: " + part.text if part.text  else "" 
            c += "* Function Call *: " + str(part.function_call) if part.function_call else ""
            c += "* Function Response *: " + str(part.function_response.response['result']) if part.function_response else ""
            print(c)
            print()

chat = Chat(model)
r = chat.toolloop('Come up with a plan to cancel all orders for customer C1 for me.', tools=tools, trace_func=print_msgs)
r

- Retrieving customer C1

Message 1, Part 1:

* Text *: Come up with a plan to cancel all orders for customer C1 for me.


Message 2, Part 1:

* Function Call *: id=None args={'customer_id': 'C1'} name='get_customer_info'

- Cancelling order O2

Message 1, Part 1:

* Function Response *: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Cancelled'}]}


Message 2, Part 1:

* Text *: OK. John Doe has two orders, O1 and O2. O1 is already shipped so I cannot cancel it. I will cancel order O2 for you.



Message 2, Part 2:

* Function Call *: id=None args={'order_id': 'O2'} name='cancel_order'


Message 1, Part 1:

* Function Response *: True


Message 2, Part 1:

* Text *: OK. I have cancelled order O2 for John Doe.




OK. I have cancelled order O2 for John Doe.<br />
<details><ul><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 257; Out: 13; Total: 270</li><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>avg_logprobs</code>: -0.025467858864710882</li><li><code>finish_reason</code>: FinishReason.STOP</li><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: OK. I have cancelled order O2 for John Doe.
</li></ul></details></li></ul></li></ul></details></li><li><code>automatic_function_calling_history</code>: </li></ul></details>